#### Apply TinyML techniques to reduce model sizes

Optimize the model using TensorFlow Lite and quantization. Convert the trained model to the TensorFlow Lite format and then apply quantization to reduce its size.

In [46]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [47]:
# Load back X_train, X_test, y_train, and y_test
X_train = np.loadtxt('train_data/X_train.txt')
X_test = np.loadtxt('train_data/X_test.txt')
y_train = np.loadtxt('train_data/y_train.txt')
y_test = np.loadtxt('train_data/y_test.txt')

In [48]:
# Feature Engineering
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Create a Deep Neural-Networks

In [49]:
# Model Selection and Training
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


#### Model Training

In [50]:
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
21/21 [==============================] - 1s 54ms/step - loss: 0.0881 - accuracy: 0.9673 - val_loss: 1.0558e-04 - val_accuracy: 1.0000
Epoch 2/10
21/21 [==============================] - 1s 44ms/step - loss: 2.6404e-05 - accuracy: 1.0000 - val_loss: 9.8665e-06 - val_accuracy: 1.0000
Epoch 3/10
21/21 [==============================] - 1s 43ms/step - loss: 5.9485e-14 - accuracy: 1.0000 - val_loss: 6.3569e-06 - val_accuracy: 1.0000
Epoch 4/10
21/21 [==============================] - 1s 42ms/step - loss: 6.0204e-14 - accuracy: 1.0000 - val_loss: 6.0123e-06 - val_accuracy: 1.0000
Epoch 5/10
21/21 [==============================] - 1s 42ms/step - loss: 6.5347e-14 - accuracy: 1.0000 - val_loss: 5.9704e-06 - val_accuracy: 1.0000
Epoch 6/10
21/21 [==============================] - 1s 42ms/step - loss: 6.5519e-14 - accuracy: 1.0000 - val_loss: 5.9654e-06 - val_accuracy: 1.0000
Epoch 7/10
21/21 [==============================] - 1s 42ms/step - loss: 6.5544e-14 - accuracy: 1.0000 - val_l

Model Evaluation

In [51]:
# Model Evaluation
predictions = model.predict(X_test_scaled)
predictions_binary = (predictions > 0.5).astype(int)

7/7 [==============================] - 0s 4ms/step


In [52]:
print(classification_report(y_test, predictions_binary))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       204
         1.0       1.00      1.00      1.00         7

    accuracy                           1.00       211
   macro avg       1.00      1.00      1.00       211
weighted avg       1.00      1.00      1.00       211



In [53]:
# Convert and save the TensorFlow model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\theor\AppData\Local\Temp\tmp3tifeuej\assets


INFO:tensorflow:Assets written to: C:\Users\theor\AppData\Local\Temp\tmp3tifeuej\assets


Save Model

In [54]:
# Save the TensorFlow Lite model to a file
model_file_path = os.path.join('models', 'dnn_model.tflite')
with open(model_file_path, "wb") as f:
    f.write(tflite_model)

Optimize the model

In [55]:
# Set optimizations (quantization)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Convert the model
tflite_model_optimize = converter.convert()

model_file_path = os.path.join('models', 'model_optimize.tflite')
with open(model_file_path, "wb") as f:
    f.write(tflite_model_optimize)

INFO:tensorflow:Assets written to: C:\Users\theor\AppData\Local\Temp\tmpq9kkh8cw\assets


INFO:tensorflow:Assets written to: C:\Users\theor\AppData\Local\Temp\tmpq9kkh8cw\assets


Evaluate the optimized model

In [60]:
# Load the optimized TensorFlow Lite model
optimized_model_path = 'models/model_optimize.tflite'
interpreter = tf.lite.Interpreter(model_path=optimized_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data
input_data = X_test_scaled[0:1].astype(np.float32)  # Take the first sample from X_test_scaled

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Convert output to binary predictions
predictions_binary = (output_data > 0.5).astype(int)

# Evaluate the predictions
print(classification_report(y_test[0:1], predictions_binary))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

